In [4]:
cd /Users/Belal/Projects/sign2text/

/Users/Belal/Projects/sign2text


In [6]:
import string
import cv2
from processing import square_pad, preprocess_for_vgg
from keras.applications import ResNet50, VGG16, InceptionV3, Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adadelta
from keras.layers import Input
from model import create_model
import argparse
import numpy as np

Using TensorFlow backend.


In [8]:
# ====== Create model for evaluation ======
# =========================================

batch_size = 32
num_channels = 3
weights = "weights/vgg16_pretrain_weights.h5"

# Create pre-trained model + classification block, with or without pre-trained weights
my_model = create_model(model="vgg16", model_weights_path=weights)

# Dictionary to convert numerical classes to alphabet
label_dict = {pos: letter
              for pos, letter in enumerate(string.ascii_uppercase)}


[INFO] loading vgg16...
[INFO] creating model.
[INFO] loading model weights.


In [9]:
my_model.compile(optimizer=Adadelta(),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

In [10]:
# define validation data generator

test_datagen = ImageDataGenerator(rescale=1.,
                                  featurewise_center=True)

test_datagen.mean = np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape(1, 1, 3)

test_generator = test_datagen.flow_from_directory(
            "./train_data/FINAL_DATA/validation/",
            target_size=(224, 224),
            batch_size=32,
            class_mode="categorical",
            color_mode="rgb",
            shuffle=False
            )

Found 374 images belonging to 26 classes.


In [34]:
score = my_model.predict_generator(test_generator, steps=12, workers=6, pickle_safe=False)

In [35]:
# evaluate the model using test generator
scores = my_model.evaluate_generator(test_generator, steps=12, workers=7, pickle_safe=False)
print("%s: %.2f%%" % (my_model.metrics_names[1], scores[1]*100))

#cvscores.append(scores[1] * 100)
#print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

acc: 97.80%


In [30]:
np.argmax(score)

0

In [33]:
np.max(score[0])

1.0

In [ ]:
from sklearn import metrics
>>> y_true = [0, 1, 2, 0, 1, 2]
>>> y_pred = [0, 2, 1, 0, 0, 1]
metrics.precision_score(y_true, y_pred, average='macro')  
metrics.recall_score(y_true, y_pred, average='micro')
metrics.f1_score(y_true, y_pred, average='weighted')  
metrics.fbeta_score(y_true, y_pred, average='macro', beta=0.5)  
metrics.precision_recall_fscore_support(y_true, y_pred, beta=0.5, average=None)
